In [46]:
%load_ext autoreload
%autoreload 2

import os
import json
import lzma
import pickle
import gc

from tqdm import tqdm

from src.datasets import IndoSum
from src.common import get_device
from src.indobart.base import get_model, get_tokenizer, get_config

import stanza
import torch
import spacy
from spacy.tokens import Doc
from spacy import displacy

import numpy as np
import nltk
import evaluate
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

from transformers import BartModel, BartConfig
from transformers import MBartForConditionalGeneration, MBartConfig
from transformers.models.bart.modeling_bart import BartAttention
from transformers.models.mbart.modeling_mbart import MBartSdpaAttention
import torch.nn as nn

import diskcache as dc

from accelerate import Accelerator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
accelerator = Accelerator()
device = accelerator.device
device

device(type='cuda')

In [3]:
# Download and set up Stanza's Indonesian NLP model
stanza.download("id")
nlp = stanza.Pipeline("id", processors="tokenize,pos,lemma,depparse", use_gpu=True, device=device)
nlp

2024-11-15 02:01:04 INFO: Downloaded file to /home/paperspace/stanza_resources/resources.json
2024-11-15 02:01:04 INFO: Downloading default packages for language: id (Indonesian) ...
2024-11-15 02:01:05 INFO: File exists: /home/paperspace/stanza_resources/id/default.zip
2024-11-15 02:01:13 INFO: Finished downloading models and saved to /home/paperspace/stanza_resources
2024-11-15 02:01:13 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-11-15 02:01:13 INFO: Downloaded file to /home/paperspace/stanza_resources/resources.json
2024-11-15 02:01:13 WARNING: Language id package default expects mwt, which has been added
2024-11-15 02:01:14 INFO: Loading these models for language: id (Indonesian):
| Processor | Package      |
----------------------------
| tokenize  | gsd          |
| mwt       | gsd          |
| pos       | gsd_charlm   |
| lemma     | gsd_nocharlm |
| depparse  | gsd_charlm   |

2024-11-15 02:01:14 INFO: Using device: cuda
2024-11-15 02:01:14 INFO: Loading: tokenize
/home/paperspace/thesis-binus/.venv/lib/python3.9/site-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more de

### Data Loading

In [4]:
indosum = IndoSum()
indosum.ds

DatasetDict({
    train: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 14262
    })
    test: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 3762
    })
    validation: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 750
    })
})

In [5]:
indosum.to_pd("train").head()

,document,id,summary
0,"Jakarta, CNN Indonesia - - Dokter Ryan Thamrin...",1501893029-lula-kamal-dokter-ryan-thamrin-saki...,Dokter Lula Kamal yang merupakan selebriti sek...
1,Selfie ialah salah satu tema terpanas di kalan...,1509072914-dua-smartphone-zenfone-baru-tawarka...,Asus memperkenalkan ZenFone generasi keempat...
2,"Jakarta, CNN Indonesia - - Dinas Pariwisata Pr...",1510613677-songsong-visit-2020-bengkulu-perkua...,Dinas Pariwisata Provinsi Bengkulu kembali men...
3,Merdeka.com - Indonesia Corruption Watch (ICW)...,1502706803-icw-ada-kejanggalan-atas-tewasnya-s...,Indonesia Corruption Watch (ICW) meminta Komis...
4,Merdeka.com - Presiden Joko Widodo (Jokowi) me...,1503039338-pembagian-sepeda-usai-upacara-penur...,Jokowi memimpin upacara penurunan bendera. Usa...


### Dependency Parsing

In [6]:
# Convert Stanza Document to spaCy Doc for Visualization
def stanza_to_spacy(doc):
    """
    Converts a Stanza-parsed document to a spaCy Doc object for dependency visualization.
    Handles root words and ensures valid head indices.
    """
    # Flatten all sentences' words, heads, and dependency relations from Stanza
    words = [word.text for sentence in doc.sentences for word in sentence.words]
    deps = [word.deprel for sentence in doc.sentences for word in sentence.words]

    # Stanza `head` is 1-based (1-indexed); convert to 0-based for spaCy and handle roots
    heads = []
    for sentence in doc.sentences:
        for word in sentence.words:
            # If the word is root, set head to its own index
            if word.head == 0:
                heads.append(word.id - 1)
            else:
                heads.append(word.head - 1)  # Convert to 0-based indexing

    # Create a spaCy Doc object using the extracted information
    spacy_doc = Doc(spacy.blank("id").vocab, words=words)
    for token, head, dep in zip(spacy_doc, heads, deps):
        token.dep_ = dep
        token.head = spacy_doc[head]

    return spacy_doc

# Build Dependency Information Matrices (DIM) for each sentence in a document
def build_dependency_matrices(document):
    """
    Build a list of Dependency Information Matrices (DIMs) for each sentence in the document.
    Each matrix represents dependency relations within a sentence.
    """
    doc = nlp(document)  # Process the document with Stanza
    matrices = []  # List to hold the DIM for each sentence in the document
    sentence_texts = []  # List to hold the raw sentences

    # Iterate over each sentence in the processed Stanza document
    for sentence in doc.sentences:
        n = len(sentence.words)  # Number of words in the sentence
        matrix = np.zeros((n, n))  # Initialize an n x n matrix with zeros

        # Populate the matrix with dependency information
        for word in sentence.words:
            if word.head > 0:  # If head is not root (head == 0 indicates root in Stanza)
                # Set a 1 for both directions (i.e., word -> head and head -> word)
                matrix[word.id - 1, word.head - 1] = 1  # word.id and word.head are 1-based indices
                matrix[word.head - 1, word.id - 1] = 1  # Make the matrix symmetric
        matrices.append(torch.tensor(matrix, dtype=torch.float32))  # Convert matrix to tensor and add to list
        sentence_texts.append(sentence.text)  # Add the raw sentence text to the list

    return list(zip(matrices, sentence_texts)), doc

# Parse and Visualize Dependencies
def visualize_dependencies(doc):
    """
    Visualizes dependencies from the Stanza-parsed document using spaCy's displacy.
    """
    # Convert Stanza output to spaCy format for visualization
    spacy_doc = stanza_to_spacy(doc)
    
    # Visualize dependencies using spaCy's displacy
    displacy.render(spacy_doc, style="dep", jupyter=True)  # Use jupyter=True in notebooks

#### Data Exploration

In [7]:
sample_doc = indosum.ds["validation"][0]['document']
sample_doc

'Ketua MPR Zulkifli Hasan menyesalkan kisruh yang terjadi antara pelaku sarana transportasi online dan tradisional. Zulkifli menyarankan adanya pertemuan bersama antara pemerintah, pelaku transportasi online dan transportasi tradisional demi meredam kisruh yang masih belum terselesaikan. Zulkifli menilai aturan yang dikeluarkan pemerintah seharusnya tidak hanya membahas tarif tapi juga mekanisme yang dapat menguntungkan semua pihak, baik pelaku transportasi online maupun tradisional. " Tidak hanya tarif tapi apa saja harus diatur. Dipanggil keduanya untuk berbicara masing-masing, musyawarah, duduk bareng kemudian dibuat aturan yang saling menguntungkan. Kan bisa saling melengkapi, negara lain bisa masa kita enggak bisa, " ucap Zulkifli di Gedung DPR, Senayan, Jakarta Pusat, Senin (27 / 3). Baca juga: Setya Novanto: Jangan Sampai Kisruh Taksi dan Ojek Online Jadi Besar Ketua Umum PAN menambahkan bahwa hal ini harus diatur karena menyangkut mata pencaharian dari masyarakat itu sendiri. M

In [8]:
sample_dim_sentence_pairs, sample_stanza_doc = build_dependency_matrices(sample_doc)
print("Dependency Information Matrices for each sentence:")
for i, (matrix, sentence) in enumerate(sample_dim_sentence_pairs, 1):
    print(f"Raw Sentence {i}:\n{sentence}")
    print(f"DIM Sentence {i}:\n{matrix}\n")

Dependency Information Matrices for each sentence:
Raw Sentence 1:
Ketua MPR Zulkifli Hasan menyesalkan kisruh yang terjadi antara pelaku sarana transportasi online dan tradisional.
DIM Sentence 1:
tensor([[0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0

### Linguistic-Guided Attention in the Encoder

In [47]:
# # Define a custom attention layer for the encoder to use the DIM during the attention calculation
# class EncoderLinguisticGuidedAttention(BartAttention):
#     def __init__(self, embed_dim, num_heads, dropout=0.0, alpha=1.0):
#         super().__init__(embed_dim, num_heads, dropout, is_decoder=False)
#         self.alpha = alpha

#     def forward(self, hidden_states, dim_matrix, **kwargs):
#         attn_output, attn_weights = super().forward(hidden_states, **kwargs)
        
#         # Apply linguistic-guided attention
#         dim_matrix = dim_matrix.to(attn_weights.device)
#         lg_attn_weights = (self.alpha * dim_matrix + torch.eye(dim_matrix.size(-1), device=dim_matrix.device)) * attn_weights
#         attn_output = torch.matmul(lg_attn_weights, hidden_states)
#         return attn_output, lg_attn_weights


# Define a custom attention layer for the encoder to use the DIM during attention calculation
class EncoderLinguisticGuidedAttention(MBartSdpaAttention):
    def __init__(self, embed_dim, num_heads, dropout=0.0, alpha=1.0):
        super().__init__(embed_dim, num_heads, dropout, is_decoder=False)
        self.alpha = alpha

    def forward(self, hidden_states, dim_matrix=None, **kwargs):
        # Compute standard attention output and weights
        attn_output, attn_weights = super().forward(hidden_states, **kwargs)

        # Apply linguistic-guided attention if dim_matrix is provided
        if dim_matrix is not None:
            dim_matrix = dim_matrix.to(attn_weights.device)  # Ensure dim_matrix is on the correct device
            lg_attn_weights = (self.alpha * dim_matrix + torch.eye(dim_matrix.size(-1), device=dim_matrix.device)) * attn_weights
            attn_output = torch.matmul(lg_attn_weights, hidden_states)
        else:
            lg_attn_weights = attn_weights  # Use standard attention weights if no dim_matrix is provided

        return attn_output, lg_attn_weights

In [48]:
# # Now, replace the encoder's attention mechanism with EncoderLinguisticGuidedAttention.
# class CustomIndoMBartWithLGA(MBartForConditionalGeneration):
#     def __init__(self, config: MBartConfig, alpha=1.0):
#         super().__init__(config)

#         # Modify encoder layers to use linguistic-guided attention
#         for layer in self.model.encoder.layers:
#             layer.self_attn = EncoderLinguisticGuidedAttention(
#                 config.d_model, config.encoder_attention_heads, config.attention_dropout, alpha=alpha
#             )


# Define a custom MBart model class with linguistic-guided attention in the encoder
class CustomIndoMBartWithLGA(MBartForConditionalGeneration):
    def __init__(self, config: MBartConfig, alpha=1.0):
        super().__init__(config)
        self.alpha = alpha

        # Replace the encoder's attention layers with EncoderLinguisticGuidedAttention
        for layer in self.model.encoder.layers:
            layer.self_attn = EncoderLinguisticGuidedAttention(
                config.d_model, config.encoder_attention_heads, config.attention_dropout, alpha=self.alpha
            )

    def forward(self, input_ids, attention_mask=None, labels=None, dim_matrix=None, **kwargs):
        # Pass dim_matrix to each encoder layer's attention if provided
        if dim_matrix is not None:
            for i, layer in enumerate(self.model.encoder.layers):
                # Set dim_matrix to be used in each layer's attention call
                layer.self_attn.dim_matrix = dim_matrix[i] if isinstance(dim_matrix, list) else dim_matrix

        # Proceed with the standard forward pass
        return super().forward(input_ids=input_ids, attention_mask=attention_mask, labels=labels, **kwargs)

### Load Model

In [11]:
tokenizer = get_tokenizer()
tokenizer


IndoNLGTokenizer(name_or_path='indobenchmark/indobart-v2', vocab_size=40004, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<mask>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	40003: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}

In [12]:
config = get_config()
config

MBartConfig {
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 1024,
  "model_type": "mbart",
  "no_repeat_ngram_size": 3,
  "no

In [13]:
pretrained_model = get_model()

In [14]:
pretrained_model

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(40004, 768, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(40004, 768, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [15]:
model = CustomIndoMBartWithLGA(config, alpha=1.0)


In [16]:
model

CustomIndoMBartWithLGA(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(40004, 768, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(40004, 768, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x MBartEncoderLayer(
          (self_attn): EncoderLinguisticGuidedAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
     

In [17]:
model.load_state_dict(get_model().state_dict(), strict=False)

<All keys matched successfully>

In [18]:
model

CustomIndoMBartWithLGA(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(40004, 768, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(40004, 768, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x MBartEncoderLayer(
          (self_attn): EncoderLinguisticGuidedAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
     

### Train Model

In [19]:
# Setup evaluation
nltk.download("punkt_tab", quiet=True)
metric = evaluate.load("rouge")

#### Preparation

In [20]:
# Set up the cache
os.makedirs(f"./results/00-indobart-dp/dim_cache", exist_ok=True)
cache_path = "./results/00-indobart-dp/dim_cache/dim_cache.xz" # lzma compressed pickle file

# Dictionary to hold all precomputed DIMs
dim_cache = {}

# Precompute DIMs and store them in the dictionary with progress tracking
def precompute_dims(documents):
    print("Precomputing Dependency Information Matrices (DIMs) for documents...")
    for i, document in enumerate(tqdm(documents, desc="Computing DIMs")):
        dim_sentence_pairs, _ = build_dependency_matrices(document)
        dim_matrices, _ = zip(*dim_sentence_pairs)
        dim_cache[i] = [dm.tolist() for dm in dim_matrices]  # Store DIMs as lists for easier serialization

# Save the entire DIM cache as a single file with progress tracking
def save_cache(filename="dim_cache.xz"):
    print(f"Saving DIM cache to {filename}...")
    # with open(filename, "wb") as f:
    #     pickle.dump(dim_cache, f)
    with lzma.open(filename, "wb") as f:
        pickle.dump(dim_cache, f)
    print("DIM cache saved successfully.")

# Load the DIM cache from a single file
def load_cache(filename="dim_cache.xz"):
    print(f"Loading DIM cache from {filename}...")
    # with open(filename, "rb") as f:
    #     cache = pickle.load(f)
    with lzma.open(filename, "rb") as f:
        cache = pickle.load(f)
    print("DIM cache loaded successfully.")
    return cache

# Main function to check cache existence, load or precompute DIMs, and save if necessary
def prepare_dim_cache(documents, cache_path="dim_cache.xz"):
    global dim_cache  # Use the global dim_cache dictionary

    if os.path.exists(cache_path):
        # If cache file exists, load it
        dim_cache = load_cache(cache_path)
    else:
        # If cache file does not exist, precompute DIMs and save to cache
        precompute_dims(documents)
        save_cache(cache_path)

# Example usage
prepare_dim_cache(indosum.ds["train"]["document"], cache_path)

Loading DIM cache from ./results/00-indobart-dp/dim_cache/dim_cache.xz...
DIM cache loaded successfully.


In [21]:
# convert dim_cache to torch tensor, use tqdm for progress tracking
for i in tqdm(dim_cache):
    # check if the DIMs are already torch tensors
    if not all(isinstance(dm, torch.Tensor) for dm in dim_cache[i]):
        # convert the DIMs to torch tensors
        dim_cache[i] = [torch.tensor(dm, dtype=torch.float32) for dm in dim_cache[i]]

100%|██████████| 14262/14262 [00:21<00:00, 650.99it/s]


In [22]:
# check the cache of the first document
dim_cache[0]

[tensor([[0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0.,

In [23]:
# Update data collator to include DIM
class CustomDataCollator(DataCollatorForSeq2Seq):
    # def __call__(self, features):
    #     batch = super().__call__(features)
        
    #     print(features)
        
    #     # if no DIMs are present, return the batch as is
    #     if "dim_matrices" not in features[0]:
    #         return batch
        
        
    #     # Flatten and pad DIMs across documents in the batch for consistent dimensions
    #     max_sentences = max(len(f["dim_matrices"]) for f in features)
    #     max_tokens = max(matrix.size(0) for f in features for matrix in f["dim_matrices"])

    #     # Initialize padded tensor for batched DIMs
    #     dim_matrices_padded = torch.zeros((len(features), max_sentences, max_tokens, max_tokens))

    #     for i, feature in enumerate(features):
    #         for j, matrix in enumerate(feature["dim_matrices"]):
    #             dim_matrices_padded[i, j, :matrix.size(0), :matrix.size(1)] = matrix

    #     batch["dim_matrices"] = dim_matrices_padded
    #     return batch


    def __call__(self, features):
        batch = super().__call__(features)
        
        # Add dim_matrices to the batch, assuming they are present in each feature
        if "dim_matrices" in features[0]:
            batch["dim_matrices"] = [feature["dim_matrices"] for feature in features]
        
        return batch



data_collator = CustomDataCollator(tokenizer=tokenizer, model=model)

In [41]:
class CustomSeq2SeqTrainer(Seq2SeqTrainer):
    # def compute_loss(self, model, inputs, return_outputs=False):
    #     # Extract dim_matrices from inputs and pass it to the model's forward method
    #     dim_matrix = inputs.pop("dim_matrices", None)
    #     outputs = model(**inputs, dim_matrix=dim_matrix)
    #     loss = outputs.loss
    #     return (loss, outputs) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        # Extract dim_matrices from inputs and pass it to the model's forward method
        dim_matrix = inputs.pop("dim_matrices", None)
        inputs["dim_matrix"] = dim_matrix  # Pass dim_matrix to the model during evaluation
        return super().prediction_step(model, inputs, prediction_loss_only, ignore_keys=ignore_keys)

In [25]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    return result

In [26]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples["document"], max_length=768, truncation=True)
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

tokenized_ds = indosum.ds.map(preprocess_function, batched=True)
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['document', 'id', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14262
    })
    test: Dataset({
        features: ['document', 'id', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3762
    })
    validation: Dataset({
        features: ['document', 'id', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 750
    })
})

In [27]:
# Function to retrieve dim_matrices from dim_cache and add it to the dataset without removing existing data
def add_dim_matrices_from_cache(examples, indices):
    # Retrieve dim_matrices for the batch using indices
    dim_matrices = [dim_cache[i] for i in indices]
    # Return the original examples along with the new dim_matrices column
    examples["dim_matrices"] = dim_matrices
    return examples

# Apply map again to add the dim_matrices column without removing existing columns
tokenized_ds["train"] = tokenized_ds["train"].map(
    add_dim_matrices_from_cache, 
    with_indices=True, 
    batched=True
)

Map:   0%|          | 0/14262 [00:00<?, ? examples/s]

In [43]:
def train_model(output_dir, per_device_batch_size, learning_rate, num_train_epochs, generation_max_length):
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir + "/checkpoint",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_batch_size,
        per_device_eval_batch_size=per_device_batch_size,
        weight_decay=0.01,
        num_train_epochs=num_train_epochs,
        fp16=True,
        predict_with_generate=True,
        generation_max_length=generation_max_length,
        log_level="info",
        logging_first_step=True,
        logging_dir=output_dir + "/logs",
        resume_from_checkpoint=True,
        save_total_limit=1,
    )

    trainer = CustomSeq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_ds["train"],
        eval_dataset=tokenized_ds["validation"],
        processing_class=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    return trainer
    
def evaluate_model(trainer):
    eval_results = trainer.evaluate(eval_dataset=tokenized_ds["test"])
    return eval_results


def train_and_evaluate(output_dir, per_device_batch_size, learning_rate, num_train_epochs, generation_max_length):
    trainer = train_model(output_dir, per_device_batch_size, learning_rate, num_train_epochs, generation_max_length)
    eval_results = evaluate_model(trainer)
    
    return trainer, eval_results


#### Training & Evaluation

Try multiple generation max length with the rest parameters fixed.
Observes the best score and the corresponding generation max length.

In [44]:
experiments = []

for i in range(1, 6):
    generation_max_length = 50 + i * 10
    experiments.append({
        "output_dir": f"./results/00-indobart-dp/model/0{i}",
        "per_device_batch_size": 8,
        "learning_rate": 3.75e-5,
        "num_train_epochs": 3,
        "generation_max_length": generation_max_length
    })

for exp in experiments:
    os.makedirs(exp["output_dir"], exist_ok=True)
    
    trainer, eval_results = train_and_evaluate(
        exp["output_dir"],
        exp["per_device_batch_size"],
        exp["learning_rate"],
        exp["num_train_epochs"],
        exp["generation_max_length"]
    )
    
    # print params and the results
    print("=== Results for experiment ===")
    print("-- Params --") 
    print(json.dumps(exp, indent=4))
    print("-- Eval results --")
    print(json.dumps(eval_results, indent=4))
    
    # save mapping between params and results
    with open(exp["output_dir"] + "/params.json", "w") as f:
        json.dump(exp, f)
    
    with open(exp["output_dir"] + "/eval_results.json", "w") as f:
        json.dump(eval_results, f)



PyTorch: setting up devices


The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using auto half precision backend
The following columns in the training set don't have a corresponding argument in `CustomIndoMBartWithLGA.forward` and have been ignored: summary, document, id, dim_matrices. If summary, document, id, dim_matrices are not expected by `CustomIndoMBartWithLGA.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 14,262
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5,349
  Number of trainable parameters = 131,543,040


ValueError: too many values to unpack (expected 2)